In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap

In [3]:
df = pd.read_csv('Crime_Data_from_2020_to_Present_CLEANED.csv')

In [4]:
df.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Part 1-2,Crm Cd,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Premis Desc,Weapon Desc,Status Desc,LOCATION,LAT,LON
0,190326475,03/01/2020 12:00:00 AM,03/01/2020 12:00:00 AM,2130,7,Wilshire,1,510,VEHICLE - STOLEN,0.0,M,O,STREET,NaN,Adult Arrest,1900 S LONGWOOD AV,34.0375,-118.3506
1,200106753,02/09/2020 12:00:00 AM,02/08/2020 12:00:00 AM,1800,1,Central,1,330,BURGLARY FROM VEHICLE,47.0,M,O,BUS STOP/LAYOVER (ALSO QUERY 124),NaN,Invest Cont,1000 S FLOWER ST,34.0444,-118.2628
2,200320258,11/11/2020 12:00:00 AM,11/04/2020 12:00:00 AM,1700,3,Southwest,1,480,BIKE - STOLEN,19.0,X,X,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,Invest Cont,1400 W 37TH ST,34.0210,-118.3002
3,200907217,05/10/2023 12:00:00 AM,03/10/2020 12:00:00 AM,2037,9,Van Nuys,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),19.0,M,O,CLOTHING STORE,NaN,Invest Cont,14000 RIVERSIDE DR,34.1576,-118.4387
4,220614831,08/18/2022 12:00:00 AM,08/17/2020 12:00:00 AM,1200,6,Hollywood,2,354,THEFT OF IDENTITY,28.0,M,H,SIDEWALK,NaN,Invest Cont,1900 TRANSIENT,34.0944,-118.3277


NUMERICAL INSIGHTS

In [5]:
# Lets check out the mean, median, and mode stats for our numerical columns
df.describe()

,DR_NO,TIME OCC,AREA,Part 1-2,Crm Cd,Vict Age,LAT,LON
count,9.646890e+05,964689.000000,964689.000000,964689.000000,964689.000000,964560.000000,964689.000000,964689.000000
mean,2.194254e+08,1338.607249,10.709620,1.405426,500.276756,29.256597,34.074142,-118.354726
std,1.276489e+07,652.260980,6.103981,0.490975,206.390212,21.934565,0.111255,0.104487
min,8.170000e+02,1.000000,1.000000,1.000000,110.000000,0.000000,33.705900,-118.667600
25%,2.106005e+08,900.000000,6.000000,1.000000,331.000000,0.000000,34.014900,-118.430900
50%,2.208049e+08,1420.000000,11.000000,1.000000,442.000000,30.000000,34.059200,-118.322900
75%,2.308146e+08,1900.000000,16.000000,2.000000,626.000000,45.000000,34.164900,-118.274000
max,2.499187e+08,2359.000000,21.000000,2.000000,956.000000,120.000000,34.334300,-118.155400


CATEGORICAL INSGIHTS

In [6]:
# Now lets see what, at a glance, what crimes are the most and least common
crime_data_summary = df['Crm Cd Desc'].value_counts()
crime_data_summary

Crm Cd Desc
VEHICLE - STOLEN                                           107284
BATTERY - SIMPLE ASSAULT                                    74267
BURGLARY FROM VEHICLE                                       60072
THEFT OF IDENTITY                                           59776
VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)     58642
                                                            ...  
GRAND THEFT / AUTO REPAIR                                       6
FIREARMS EMERGENCY PROTECTIVE ORDER (FIREARMS EPO)              5
DISHONEST EMPLOYEE ATTEMPTED THEFT                              4
FIREARMS RESTRAINING ORDER (FIREARMS RO)                        4
TRAIN WRECKING                                                  1
Name: count, Length: 139, dtype: int64

We can see right away that auto theft is by far the most common crime in LA over the past 4 years. Takes dive a little deeper and see if the COVID lockdown played a factor in this. 

For the sake of the analysis, we will consider peak COVID lockdown from the time of May 2020 - May 2022. This is an important factor in auto theft because when you are contained to being at home you are not driving your car, or checking on it for that matter. Which I hypothesize made it a lot easier for burglars to steal them.

(We will also use the Date Occured Column for this analysis)

In [7]:
# First lets make sure that our Date Occured column is in Datetime format
df['DATE OCC'] = pd.to_datetime(df['DATE OCC'])
df['Date Rptd'] = pd.to_datetime(df['Date Rptd'])
df.head()

/var/folders/z2/sjjs462s7zzf2bgvh3gnrknh0000gn/T/ipykernel_95512/151245219.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['DATE OCC'] = pd.to_datetime(df['DATE OCC'])
/var/folders/z2/sjjs462s7zzf2bgvh3gnrknh0000gn/T/ipykernel_95512/151245219.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date Rptd'] = pd.to_datetime(df['Date Rptd'])


,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Part 1-2,Crm Cd,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Premis Desc,Weapon Desc,Status Desc,LOCATION,LAT,LON
0,190326475,2020-03-01,2020-03-01,2130,7,Wilshire,1,510,VEHICLE - STOLEN,0.0,M,O,STREET,NaN,Adult Arrest,1900 S LONGWOOD AV,34.0375,-118.3506
1,200106753,2020-02-09,2020-02-08,1800,1,Central,1,330,BURGLARY FROM VEHICLE,47.0,M,O,BUS STOP/LAYOVER (ALSO QUERY 124),NaN,Invest Cont,1000 S FLOWER ST,34.0444,-118.2628
2,200320258,2020-11-11,2020-11-04,1700,3,Southwest,1,480,BIKE - STOLEN,19.0,X,X,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,Invest Cont,1400 W 37TH ST,34.0210,-118.3002
3,200907217,2023-05-10,2020-03-10,2037,9,Van Nuys,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),19.0,M,O,CLOTHING STORE,NaN,Invest Cont,14000 RIVERSIDE DR,34.1576,-118.4387
4,220614831,2022-08-18,2020-08-17,1200,6,Hollywood,2,354,THEFT OF IDENTITY,28.0,M,H,SIDEWALK,NaN,Invest Cont,1900 TRANSIENT,34.0944,-118.3277


In [8]:
# Now we can split the rows into two tables, pandemic and non-pandemic
start_date = '2020-05-01'
end_date = '2022-05-01'

covid_lockdown = df[(df['DATE OCC'] >= start_date) & (df['DATE OCC'] < end_date)]
non_covid = df[(df['DATE OCC'] < start_date) | (df['DATE OCC'] >= end_date)]

In [9]:
# Now we can check the crime stats for our new tables
covid_lockdown_crime = covid_lockdown['Crm Cd Desc'].value_counts()
covid_lockdown_crime

Crm Cd Desc
VEHICLE - STOLEN                                           46730
BATTERY - SIMPLE ASSAULT                                   32335
VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)    26862
BURGLARY FROM VEHICLE                                      26219
BURGLARY                                                   25407
                                                           ...  
PICKPOCKET, ATTEMPT                                            2
TELEPHONE PROPERTY - DAMAGE                                    2
FAILURE TO DISPERSE                                            2
FIREARMS EMERGENCY PROTECTIVE ORDER (FIREARMS EPO)             2
FIREARMS RESTRAINING ORDER (FIREARMS RO)                       2
Name: count, Length: 134, dtype: int64

In [10]:
non_covid_crime = non_covid['Crm Cd Desc'].value_counts()
non_covid_crime

Crm Cd Desc
VEHICLE - STOLEN                 60554
BATTERY - SIMPLE ASSAULT         41932
THEFT OF IDENTITY                36204
BURGLARY FROM VEHICLE            33853
BURGLARY                         32233
                                 ...  
LYNCHING - ATTEMPTED                 2
BIGAMY                               2
BIKE - ATTEMPTED STOLEN              1
THEFT, COIN MACHINE - ATTEMPT        1
TRAIN WRECKING                       1
Name: count, Length: 139, dtype: int64

Despite my thoughts, we can see that the same types of crimes actually occured during peak COVID times and pre/post pandemic

In [11]:
# Quick glance at what neighborhoods have the most and least amount of crime (We will plot this on a map later)
neighborhood_summary = df['AREA NAME'].value_counts()
neighborhood_summary

AREA NAME
Central        66039
77th Street    60087
Pacific        56482
Southwest      54791
Hollywood      50259
N Hollywood    49049
Southeast      48387
Olympic        48163
Newton         47497
Wilshire       46177
Rampart        44843
West LA        43978
Van Nuys       41187
Northeast      41131
West Valley    40783
Devonshire     40017
Topanga        39848
Harbor         39827
Mission        38594
Hollenbeck     35479
Foothill       32071
Name: count, dtype: int64

In [12]:
# A basic Heat Map that shows the areas in LA where Crime is the hottest. We will dig deeper into this later
# locations = df[['LAT', 'LON']].dropna().values.tolist()

# m = folium.Map(location=[34.0522, -118.2437], zoom_start=10) # Central location of LA
# HeatMap(locations).add_to(m)
# m.save('Basic_Crime_Heatmap.html')

TEMPORAL INSIGHTS

For temporal stats, I am interested in seeing how crimes change by season, by month, day of the week, and year over year. Is the city getting safer or more dangerous? Are certain neighborhoods getting more dangerous over time? Are there certain days where some neighborhoods are more dangerous than others?

These are all questions we will answer in this section!

In [13]:
df['Year'] = df['DATE OCC'].dt.year
df['Month'] = df['DATE OCC'].dt.month_name()
df['Day'] = df['DATE OCC'].dt.day
df['Day of Week'] = df['DATE OCC'].dt.day_name()

In [14]:
df

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Part 1-2,Crm Cd,Crm Cd Desc,Vict Age,...,Premis Desc,Weapon Desc,Status Desc,LOCATION,LAT,LON,Year,Month,Day,Day of Week
0,190326475,2020-03-01,2020-03-01,2130,7,Wilshire,1,510,VEHICLE - STOLEN,0.0,...,STREET,NaN,Adult Arrest,1900 S LONGWOOD AV,34.0375,-118.3506,2020,March,1,Sunday
1,200106753,2020-02-09,2020-02-08,1800,1,Central,1,330,BURGLARY FROM VEHICLE,47.0,...,BUS STOP/LAYOVER (ALSO QUERY 124),NaN,Invest Cont,1000 S FLOWER ST,34.0444,-118.2628,2020,February,8,Saturday
2,200320258,2020-11-11,2020-11-04,1700,3,Southwest,1,480,BIKE - STOLEN,19.0,...,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,Invest Cont,1400 W 37TH ST,34.0210,-118.3002,2020,November,4,Wednesday
3,200907217,2023-05-10,2020-03-10,2037,9,Van Nuys,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),19.0,...,CLOTHING STORE,NaN,Invest Cont,14000 RIVERSIDE DR,34.1576,-118.4387,2020,March,10,Tuesday
4,220614831,2022-08-18,2020-08-17,1200,6,Hollywood,2,354,THEFT OF IDENTITY,28.0,...,SIDEWALK,NaN,Invest Cont,1900 TRANSIENT,34.0944,-118.3277,2020,August,17,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964684,242004546,2024-01-16,2024-01-16,1510,20,Olympic,2,624,BATTERY - SIMPLE ASSAULT,80.0,...,BUS STOP,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",Invest Cont,HOBART BL,34.0690,-118.3054,2024,January,16,Tuesday
964685,240710284,2024-07-24,2024-07-23,1400,7,Wilshire,1,510,VEHICLE - STOLEN,0.0,...,STREET,NaN,Invest Cont,4000 W 23RD ST,34.0362,-118.3284,2024,July,23,Tuesday
964686,240104953,2024-01-15,2024-01-15,100,1,Central,2,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),0.0,...,HOTEL,UNKNOWN WEAPON/OTHER WEAPON,Invest Cont,1300 W SUNSET BL,34.0685,-118.2460,2024,January,15,Monday
964687,241711348,2024-07-19,2024-07-19,757,17,Devonshire,2,888,TRESPASSING,0.0,...,MTA - ORANGE LINE - CHATSWORTH,NaN,Invest Cont,10000 OLD DEPOT PLAZA RD,34.2500,-118.5990,2024,July,19,Friday
